Option chains
=======

In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 4002, clientId=13)

<IB connected to 127.0.0.1:4002 clientId=13>

Suppose we want to find the options on the SPX, with the following conditions:

* Use the next three monthly expiries;
* Use strike prices within +- 20 dollar of the current SPX value;
* Use strike prices that are a multitude of 5 dollar.

To get the current market value, first create a contract for the underlyer (the S&P 500 index):

In [2]:
spx = Index('SPX', 'CBOE')
ib.qualifyContracts(spx)

[Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX')]

To avoid issues with market data permissions, we'll use delayed data:

In [3]:
ib.reqMarketDataType(4)

Then get the ticker. Requesting a ticker can take up to 11 seconds.

In [4]:
[ticker] = ib.reqTickers(spx)
ticker

Ticker(contract=Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX'), time=datetime.datetime(2021, 10, 20, 1, 33, 12, 657570, tzinfo=datetime.timezone.utc), marketDataType=3, bid=-1.0, bidSize=0.0, ask=-1.0, askSize=0.0, last=4520.22, lastSize=1.0, high=4520.4, low=4496.41, close=4486.46)

Take the current market value of the ticker:

In [5]:
spxValue = ticker.marketPrice()
spxValue

4486.46

The following request fetches a list of option chains:

In [6]:
chains = ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)

util.df(chains)

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,SMART,416904,SPX,100,"[20211118, 20211216, 20220120, 20220217, 20220...","[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700..."
1,CBOE,416904,SPXW,100,"[20211020, 20211022, 20211025, 20211027, 20211...","[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700..."
2,SMART,416904,SPXW,100,"[20211020, 20211022, 20211025, 20211027, 20211...","[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700..."
3,CBOE,416904,SPX,100,"[20211118, 20211216, 20220120, 20220217, 20220...","[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700..."


These are four option chains that differ in ``exchange`` and ``tradingClass``. The latter is 'SPX' for the monthly and  'SPXW' for the weekly options. Note that the weekly expiries are disjoint from the monthly ones, so when interested in the weekly options the monthly options can be added as well.

In this case we're only interested in the monthly options trading on SMART:

In [7]:
chain = next(c for c in chains if c.tradingClass == 'SPX' and c.exchange == 'SMART')
chain

OptionChain(exchange='SMART', underlyingConId='416904', tradingClass='SPX', multiplier='100', expirations=['20211118', '20211216', '20220120', '20220217', '20220317', '20220413', '20220519', '20220616', '20220714', '20220818', '20220915', '20221020', '20221117', '20221215', '20230119', '20230615', '20231214'], strikes=[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700.0, 800.0, 900.0, 1000.0, 1100.0, 1150.0, 1200.0, 1225.0, 1250.0, 1275.0, 1300.0, 1325.0, 1350.0, 1375.0, 1400.0, 1425.0, 1450.0, 1475.0, 1500.0, 1525.0, 1550.0, 1575.0, 1600.0, 1625.0, 1650.0, 1675.0, 1700.0, 1725.0, 1750.0, 1775.0, 1800.0, 1825.0, 1850.0, 1875.0, 1900.0, 1925.0, 1950.0, 1975.0, 2000.0, 2025.0, 2050.0, 2075.0, 2100.0, 2125.0, 2150.0, 2175.0, 2200.0, 2225.0, 2250.0, 2275.0, 2300.0, 2325.0, 2350.0, 2375.0, 2400.0, 2425.0, 2450.0, 2475.0, 2500.0, 2525.0, 2550.0, 2575.0, 2600.0, 2625.0, 2650.0, 2675.0, 2700.0, 2725.0, 2750.0, 2775.0, 2800.0, 2825.0, 2850.0, 2875.0, 2900.0, 2925.0, 2950.0, 2975.0, 3000.0, 3025.0, 

What we have here is the full matrix of expirations x strikes. From this we can build all the option contracts that meet our conditions:

In [8]:
strikes = [strike for strike in chain.strikes
        if strike % 5 == 0
        and spxValue - 20 < strike < spxValue + 20]
expirations = sorted(exp for exp in chain.expirations)[:3]
rights = ['P', 'C']

contracts = [Option('SPX', expiration, strike, right, 'SMART', tradingClass='SPX')
        for right in rights
        for expiration in expirations
        for strike in strikes]

contracts = ib.qualifyContracts(*contracts)
len(contracts)

48

In [9]:
contracts[0]

Option(conId=498436925, symbol='SPX', lastTradeDateOrContractMonth='20211118', strike=4470.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   211119P04470000', tradingClass='SPX')

Now to get the market data for all options in one go:

In [10]:
tickers = ib.reqTickers(*contracts)

tickers[0]

Error 10090, reqId 101: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Option(conId=521453596, symbol='SPX', lastTradeDateOrContractMonth='20220120', strike=4505.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   220121C04505000', tradingClass='SPX')
Error 10090, reqId 99: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL, contract: Option(conId=521453590, symbol='SPX', lastTradeDateOrContractMonth='20220120', strike=4495.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   220121C04495000', tradingClass='SPX')
Error 10090, reqId 100: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/T

Ticker(contract=Option(conId=498436925, symbol='SPX', lastTradeDateOrContractMonth='20211118', strike=4470.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   211119P04470000', tradingClass='SPX'))

Error 10090, reqId 57: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL
Error 10090, reqId 58: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL
Error 10090, reqId 59: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL
Error 10090, reqId 61: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL
Error 10090, reqId 60: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL
Error 10090, reqId 67: Part of requested market data is not subscribed. Sub

The option greeks are available from the ``modelGreeks`` attribute, and if there is a bid, ask resp. last price available also from ``bidGreeks``, ``askGreeks`` and ``lastGreeks``. For streaming ticks the greek values will be kept up to date to the current market situation.

In [11]:
ib.disconnect()